# 11. A Pythonic Object

Thanks to the Python Data Model, your user-defined types can behave as naturally as the built-in types. And this can be accomplished without inheritance, in the spirit of _duck typing_: you just implement the methods needed for your objects to behave as expected.

## Object Representations

Every object-oriented language has at least one standard way of getting a string repre‐ sentation from any object. Python has two:

`repr()`
    Return a string representing the object as the developer wants to see it. It’s what you get when the Python console or a debugger shows an object.

`str()`
    Return a string representing the object as the user wants to see it. It’s what you get when you `print()` an object.

## Vector Class Redux

In order to demonstrate the many methods used to generate object representations, we’ll use a `Vector2d` class similar to the one we saw before.

In [15]:
from array import array
import math

class Vector2D:
    typecode = 'd'

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (
            bytes( [ord(self.typecode)]) +
            bytes( array(self.typecode, self))
                )
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

## classmethod Versus staticmethod

The `classmethod` decorator is not mentioned in the Python tutorial, and neither is `staticmethod`. Anyone who has learned OO in Java may wonder why Python has both of these decorators and not just one of them.

## Formatted Displays

The f-strings, the `format()` built-in function, and the `str.format()` method delegate the actual formatting to each type by calling their `.__format__(format_spec)` method. The `format_spec` is a formatting specifier, which is either:

+ The second argument in `format(my_obj, format_spec)` or
+ Whatever appears after the colon in a replacement field delimited with `{}` inside an f-string or the `fmt` in `fmt.str.format()`

In [16]:
brl = 1 / 4.82
brl

0.20746887966804978

In [17]:
format(brl, '0.4f')

'0.2075'

In [18]:
'1 BRL = {rate:0.2f} USD'.format(rate=brl)

'1 BRL = 0.21 USD'

In [19]:
f'1 USD = {1 / brl:0.2f} BRL'

'1 USD = 4.82 BRL'

A few built-in types have their own presentation codes in the Format Specification Mini-Language. For example—among several other codes—the `int` type supports `b` and `x` for base 2 and base 16 output, respectively, while `float` implements `f` for a fixed-point display and `%` for a percentage display:

In [20]:
format(42, 'b')

'101010'

In [21]:
format(2/3, '.1%')

'66.7%'

In [22]:
from datetime import datetime
now = datetime.now()
format(now, '%H:%M:%S')

'10:36:49'

In [23]:
"It's now {:%I:%M %p}".format(now)

"It's now 10:36 AM"

In [24]:
v1 = Vector2D(3, 4)
format(v1)

'(3.0, 4.0)'

In [26]:
try:
    print(format(v1, '.3f'))
except Exception as e: print(f"{e=}")

e=TypeError('unsupported format string passed to Vector2D.__format__')


In [27]:
def __format__(self, fmt_spec=''):
    components = (format(c, fmt_spec) for c in self)
    return '({}, {})'.format(*components)

Vector2D.__format__ = __format__

In [30]:
v1 = Vector2D(3, 4)
format(v1)

'(3.0, 4.0)'

In [31]:
try:
    print(format(v1, '.3f'))
except Exception as e: print(f"{e=}")

(3.000, 4.000)


In [33]:
def angle(self):
    return math.atan2(self.y, self.x)

Vector2D.angle = angle

In [38]:
def __format__(self, fmt_spec=''):
    if fmt_spec.endswith('p'):
        fmt_spec = fmt_spec[:-1]
        coords = (abs(self), self.angle())
        outer_fmt = '<{}, {}>'
    else:
        coords = self
        outer_fmt = '({}, {})'
    components = (format(c, fmt_spec) for c in coords)
    return outer_fmt.format(*components)

Vector2D.__format__ = __format__

In [39]:
format(Vector2D(1, 1), 'p')

'<1.4142135623730951, 0.7853981633974483>'

In [41]:
format(Vector2D(1, 1), '.3ep')

'<1.414e+00, 7.854e-01>'

In [42]:
format(Vector2D(1, 1), '0.5fp')

'<1.41421, 0.78540>'

## A Hashable Vector2D

As defined, so far our Vector2d instances are unhashable, so we can’t put them in a `set`:

In [43]:
try:
    v1 = Vector2D(3,4)
    print(hash(v1))
except Exception as e:
    print(f"{e=}")

try:
    v1 = Vector2D(3,4)
    print(set( [v1] ))
except Exception as e:
    print(f"{e=}")

e=TypeError("unhashable type: 'Vector2D'")
e=TypeError("unhashable type: 'Vector2D'")


To make a Vector2D hashable, we must implement `__hash__` (`__eq__` is also required, and we already have it). We also need to make vector instances immutable, as we’ve seen before.

Right now, anyone can do `v1.x = 7`, and there is nothing in the code to suggest that changing a `Vector2D` is forbidden.

In [44]:
class Vector2d:
    typecode = 'd'

    def __init__(self, x, y): 
        self.__x = float(x) 
        self.__y = float(y)

    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    def __iter__(self):
        return (i for i in (self.x, self.y))

Now that our vectors are reasonably safe from accidental mutation, we can imple‐ ment the `__hash__` method. It should return an int and ideally take into account the hashes of the object attributes that are also used in the `__eq__` method, because objects that compare equal should have the same hash. The `__hash__ `special method documentation suggests computing the hash of a tuple with the components, so that’s what we do in examples.

In [45]:
def __hash__(self):
    return hash((self.x, self.y))

Vector2D.__hash__ = __hash__

In [46]:
v1 = Vector2D(3, 4)
v2 = Vector2D(3.1, 4.2)
print(f"{hash(v1)=}, {hash(v2)=}")

hash(v1)=1079245023883434373, hash(v2)=1994163070182233067


In [47]:
print(f"{v1=}, {v2=}")

v1=Vector2D(3.0, 4.0), v2=Vector2D(3.1, 4.2)


## Supporting Positional Pattern Matching

To make Vector2d work with positional patterns, we need to add a class attribute named `__match_args__` , listing the instance attributes in the order they will be used for positional pattern matching:

```python
class Vector2D:
    __match_args__ = ('x', 'y')

    # etc ...
```


## Complete Listing of Vector2D, Version3

In [48]:
"""
A two-dimensional vector class
    >>> v1 = Vector2d(3, 4)
    >>> print(v1.x, v1.y)
    3.0 4.0
    >>> x, y = v1
    >>> x, y
    (3.0, 4.0)
    >>> v1
    Vector2d(3.0, 4.0)
    >>> v1_clone = eval(repr(v1))
    >>> v1 == v1_clone
    True
    >>> print(v1)
    (3.0, 4.0)
    >>> octets = bytes(v1)
    >>> octets
    b'd\\x00\\x00\\x00\\x00\\x00\\x00\\x08@\\x00\\x00\\x00\\x00\\x00\\x00\\x10@'
    >>> abs(v1)
    5.0
    >>> bool(v1), bool(Vector2d(0, 0))
    (True, False)

Test of ``.frombytes()`` class method:
    >>> v1_clone = Vector2d.frombytes(bytes(v1))
    >>> v1_clone
    Vector2d(3.0, 4.0)
    >>> v1 == v1_clone
    True

Tests of ``format()`` with Cartesian coordinates:
    >>> format(v1)
    '(3.0, 4.0)'
    >>> format(v1, '.2f')
    '(3.00, 4.00)'
    >>> format(v1, '.3e')
    '(3.000e+00, 4.000e+00)'

Tests of the ``angle`` method::
    >>> Vector2d(0, 0).angle()
    0.0
    >>> Vector2d(1, 0).angle()
    0.0
    >>> epsilon = 10**-8
    >>> abs(Vector2d(0, 1).angle() - math.pi/2) < epsilon
    True
    >>> abs(Vector2d(1, 1).angle() - math.pi/4) < epsilon
    True

Tests of ``format()`` with polar coordinates:
    >>> format(Vector2d(1, 1), 'p')  # doctest:+ELLIPSIS
    '<1.414213..., 0.785398...>'
    >>> format(Vector2d(1, 1), '.3ep')
    '<1.414e+00, 7.854e-01>'
    >>> format(Vector2d(1, 1), '0.5fp')
    '<1.41421, 0.78540>'

Tests of `x` and `y` read-only properties:
    >>> v1.x, v1.y
    (3.0, 4.0)
    >>> v1.x = 123
    Traceback (most recent call last):
      ...
    AttributeError: can't set attribute 'x'

Tests of hashing:
    >>> v1 = Vector2d(3, 4)
    >>> v2 = Vector2d(3.1, 4.2)
    >>> len({v1, v2})
    2
"""

from array import array
import math

class Vector2d:
    __match_args__ = ('x', 'y')

    typecode = 'd'

    def __init__(self, x, y): 
        self.__x = float(x) 
        self.__y = float(y)

    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
            bytes(array(self.typecode, self))) 
    
    def __eq__(self, other):
        return tuple(self) == tuple(other) 

    def __hash__(self):
        return hash((self.x, self.y)) 

    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self)) 

    def angle(self):
        return math.atan2(self.y, self.x)
    
    def __format__(self, fmt_spec=''): 
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords) 
        return outer_fmt.format(*components)
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode) 
        return cls(*memv)

In [49]:
v1 = Vector2d(3, 4)
print(v1.x, v1.y)

3.0 4.0


In [50]:
x, y = v1
x, y

(3.0, 4.0)

In [51]:
v1

Vector2d(3.0, 4.0)

In [52]:
v1_clone = eval(repr(v1))

In [53]:
v1 == v1_clone

True

In [54]:
print(v1)

(3.0, 4.0)


In [56]:
octets = bytes(v1)
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [57]:
abs(v1)

5.0

In [58]:
bool(v1), bool(Vector2d(0, 0))

(True, False)

In [60]:
v1_clone = Vector2d.frombytes(bytes(v1))
v1_clone

Vector2d(3.0, 4.0)

In [61]:
v1 == v1_clone

True

In [62]:
format(v1)

'(3.0, 4.0)'

In [63]:
format(v1, '.2f')

'(3.00, 4.00)'

In [64]:
format(v1, '.3e')

'(3.000e+00, 4.000e+00)'

In [65]:
Vector2d(0, 0).angle()

0.0

In [66]:
Vector2d(1, 0).angle()

0.0

In [67]:
epsilon = 10**-8
abs(Vector2d(0, 1).angle() - math.pi/2) < epsilon

True

In [68]:
abs(Vector2d(1, 1).angle() - math.pi/4) < epsilon

True

In [ ]:
format(Vector2d(1, 1), 'p')

'<1.4142135623730951, 0.7853981633974483>'

In [70]:
format(Vector2d(1, 1), '.3ep')

'<1.414e+00, 7.854e-01>'

In [71]:
format(Vector2d(1, 1), '0.5fp')

'<1.41421, 0.78540>'

In [72]:
v1.x, v1.y

(3.0, 4.0)

In [73]:
try:
    v1.x = 123
except Exception as e:
    print(f"{e=}")

e=AttributeError("property 'x' of 'Vector2d' object has no setter")


In [74]:
v1 = Vector2d(3, 4)
v2 = Vector2d(3.1, 4.2)
len( {v1, v2} )

2

## Private and "Protected" Attributes in Python

In Python, there is no way to create private variables like there is with the `private` modifier in Java. What we have in Python is a simple mechanism to prevent acciden‐ tal overwriting of a “private” attribute in a subclass.

If you name an instance attribute in the form `__mood` (two leading underscores and zero or at most one trailing underscore), Python stores the name in the instance `__dict__` prefixed with a leading underscore and the class name, so in the `Dog` class, `__mood` becomes `_Dog__mood`, and in `Beagle` it’s `_Beagle__mood`. This language feature goes by the lovely name of _name mangling_.

In [75]:
v1 = Vector2d(3, 4)
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

In [76]:
v1._Vector2d__x

3.0

## Saving Memory with __slots__

By default, Python stores the attributes of each instance in a `dict` named `__dict__`.

In [78]:
class Pixel:
    __slots__ = ('x', 'y')

p = Pixel()

try:
    print(f"{p.__dict__=}")
except Exception as e:
    print(f"{e=}")

e=AttributeError("'Pixel' object has no attribute '__dict__'")


In [79]:
try:
    p.x = 10
    p.y = 20
    p.color = 'red'
except Exception as e:
    print(f"{e=}")    

e=AttributeError("'Pixel' object has no attribute 'color'")


In [80]:
class OpenPixel(Pixel): # subclass of Pixel
    pass

In [81]:
op = OpenPixel()
op.__dict__

{}

In [82]:
op.x = 8
op.__dict__

{}

In [83]:
op.x

8

In [84]:
op.color = 'green'
op.__dict__

{'color': 'green'}

In [85]:
class ColorPixel(Pixel):
    __slots__ = ('color', )

try:
    cp = ColorPixel()
    print(f"{cp.__dict__=}")
except Exception as e:
    print(f"{e=}")

e=AttributeError("'ColorPixel' object has no attribute '__dict__'")


In [87]:
try:
    cp.x = 2
    cp.color = 'blue'
    cp.flavor = 'banana'
except Exception as e:
    print(f"{e=}")

e=AttributeError("'ColorPixel' object has no attribute 'flavor'")


## Overriding Class Attributes

The default `Vector2d.typecode` is `'d'`, meaning each vector component will be rep‐ resented as an 8-byte double precision float when exporting to `bytes`. If we set the typecode of a `Vector2d` instance to `'f'` prior to exporting, each component will be exported as a 4-byte single precision `float`.

In [88]:
v1 = Vector2d(1.1, 2.2)
print(f"{v1=}")

v1=Vector2d(1.1, 2.2)


In [89]:
dumpd = bytes(v1)
dumpd

b'd\x9a\x99\x99\x99\x99\x99\xf1?\x9a\x99\x99\x99\x99\x99\x01@'

In [90]:
v1.typecode = 'f'
dumpf = bytes(v1)
dumpf

b'f\xcd\xcc\x8c?\xcd\xcc\x0c@'

In [91]:
len(dumpf)

9

In [92]:
Vector2d.typecode

'd'

In [93]:
class ShortVector2d(Vector2d):
    typecode = 'f'

sv = ShortVector2d(1/11, 1/27)
sv

ShortVector2d(0.09090909090909091, 0.037037037037037035)

In [94]:
len(bytes(sv))

9